# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

__Мягкий дедлайн: 24.09 23:59__   
__Жесткий дедлайн: 27.09 23:59__

### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в почте или поисковой строке. За дополнительные баллы полученную систему нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex) или аналогов. В этой домашке вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей: основной и бонусной. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание и штрафы

Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по __одному__ баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код.

При сдаче зададания в anytask вам будет необходимо сдать весь код, а если вы возьметесь за бонусную часть, то еще отчет и видео с демонстрацией вашего UI. За основную часть можно получить до __10-ти__ баллов, а за бонусную – до __5-ти__ баллов.

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd

emails = pd.read_csv('emails.csv')
len(emails)

517401

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (2 балла).__ Очистите корпус текстов по вашему усмотрению и объясните свой выбор. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст.

In [2]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
import string
import nltk
import ast
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

def clean_and_tokenize(text: str) -> list:
    """
    1. Убираем заголовки до первой пустой строки.
    2. Удаляем URL-ы и email-адреса.
    3. Убираем пунктуацию.
    4. Токенизируем по словам.
    5. Приводим слова к нижнему регистру.
    """
    # 1. Отделяем тело письма
    parts = re.split(r'\r?\n\r?\n', text, maxsplit=1)
    body = parts[1] if len(parts) > 1 else parts[0]
    
    # 2. Удаляем URL и email
    body = re.sub(r'http[s]?://\S+', ' ', body)
    body = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w{2,}\b', ' ', body)
    
    # 3. Убираем пунктуацию
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    no_punct = body.translate(translator)
    
    # 4. Токенизация и приведение к нижнему регистру
    tokens = word_tokenize(no_punct, language='english')
    tokens = [tok.lower() for tok in tokens if tok.isalpha()]
    
    return tokens


import json
import os

if not os.path.exists("corpus.json"):
    emails = pd.read_csv('emails.csv')
    emails['cleaned_message'] = emails['message'].apply(clean_and_tokenize)
    corpus = list(emails['cleaned_message'].values)
    with open("corpus.json", "w", encoding="utf-8") as f:
        json.dump(corpus, f, ensure_ascii=False, indent=2)

# 2. Прочитать из файла (десериализация из JSON)
with open("corpus.json", "r", encoding="utf-8") as f:
    corpus = json.load(f)

corpus

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/artembabak/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


[['here', 'is', 'our', 'forecast'],
 ['traveling',
  'to',
  'have',
  'a',
  'business',
  'meeting',
  'takes',
  'the',
  'fun',
  'out',
  'of',
  'the',
  'trip',
  'especially',
  'if',
  'you',
  'have',
  'to',
  'prepare',
  'a',
  'presentation',
  'i',
  'would',
  'suggest',
  'holding',
  'the',
  'business',
  'plan',
  'meetings',
  'here',
  'then',
  'take',
  'a',
  'trip',
  'without',
  'any',
  'formal',
  'business',
  'meetings',
  'i',
  'would',
  'even',
  'try',
  'and',
  'get',
  'some',
  'honest',
  'opinions',
  'on',
  'whether',
  'a',
  'trip',
  'is',
  'even',
  'desired',
  'or',
  'necessary',
  'as',
  'far',
  'as',
  'the',
  'business',
  'meetings',
  'i',
  'think',
  'it',
  'would',
  'be',
  'more',
  'productive',
  'to',
  'try',
  'and',
  'stimulate',
  'discussions',
  'across',
  'the',
  'different',
  'groups',
  'about',
  'what',
  'is',
  'working',
  'and',
  'what',
  'is',
  'not',
  'too',
  'often',
  'the',
  'presenter',

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат для финального пользователя будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы заметите, что из-за этого падает качество самого текса, то можете удалить все небуквенные символы на этапе токенизации.

## Общая схема решения

Мы хотим сделать систему, которая будет ускорять набор текста, советуя подходящие продолжения. Для подсказки следующего слова мы будем использовать n-граммную модель. Так как n-граммная модель работает с целыми словами, а советы мы хотим давать в риал-тайме даже когда слово еще не дописано, сперва надо научиться дополнять слово до целого.

## Дополнение слова

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова не больше чем за $O(n + mk)$, где $m$ - число подходящих слов, а $k$ – длина суффикса.

__Задание 2 (2 балла).__ Допишите префиксное дерево для поиска слов по префиксу.

Использовал псевдокод отсюда https://www.baeldung.com/cs/tries-prefix-trees

In [4]:
from typing import List
from collections import deque
from typing import Set, Dict

class PrefixTreeNode:
    def __init__(self):
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

    def get_descendants(self, prefix) -> List[str]:
        results: List[str] = []

        def dfs(current, path):
            if current.is_end_of_word:
                results.append(prefix + "".join(path))
            for ch, child in current.children.items():
                dfs(child, path + [ch])

        dfs(self, [])
        return results

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()

        for word in vocabulary:
            self._insert(word)

    def _insert(self, word: str) -> None:
        node = self.root
        for c in word:
            node = node.children.setdefault(c, PrefixTreeNode())
        node.is_end_of_word = True

    def search_prefix(self, prefix) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """
        
        node = self.root
        
        for ch in prefix:
            if ch not in node.children:
                return []
            node = node.children[ch]
        return node.get_descendants(prefix)

In [5]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его __встречаемости в корпусе__.

__Задание 3 (2 балла).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [31]:
from collections import Counter
from nltk.corpus import wordnet
import numpy as np

class WordCompletor:
    def __init__(self, corpus, min_freq = 0):
        """
        corpus: list – корпус текстов
        """
        flat_words = (word for doc in corpus for word in doc)
        freq = Counter(flat_words)
        freq = Counter({w: c for w, c in freq.items() if c >= min_freq})

        self.sum_freqs = sum(freq.values())
        self.voc = freq
        self.prefix_tree = PrefixTree(self.voc.keys())

    def get_words_and_probs(self, prefix: str):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words, probs = [], []
        words = self.prefix_tree.search_prefix(prefix)
        for word in words:
            probs.append(self.voc[word])
        probs = np.array(probs)
        probs = list(probs / self.sum_freqs)
        return words, probs

In [32]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему следующее слово (или несколько) с учетом дописанного. Для этого мы воспользуемся n-граммной моделью.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

$P(w_i \mid w_{i-1}, \dots, w_{i-n})$ оценивается по частоте встречаемости n-граммы.   

__Задание 4 (2 балла).__ Напишите класс для n-граммной модели. Никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [33]:
from collections import defaultdict
import ast
import os
import pickle

class NGramLanguageModel:
    def __init__(self, corpus, n, corpus_name):
        self.n = n
        self.prefix_counters = defaultdict(Counter)
        if os.path.exists(f"prefix_counters_{n}_{corpus_name}.pkl"):
            with open(f"prefix_counters_{n}_{corpus_name}.pkl", "rb") as f:
                self.prefix_counters = pickle.load(f)
        else:
            for text in corpus:
                if isinstance(text, str):
                    text = ast.literal_eval(text) 
                padded = ['<s>'] * (n-1) + text + ['<e>']
                for i in range(len(text)):
                    prefix = tuple(padded[i:i+n])
                    next_word = padded[i+n]
                    self.prefix_counters[prefix][next_word] += 1

            with open(f"prefix_counters_{n}_{corpus_name}.pkl", "wb") as f:
                pickle.dump(self.prefix_counters, f)
                
        self.prefix_totals = {
            prefix: sum(counter.values())
            for prefix, counter in self.prefix_counters.items()
        }

        

    def get_next_words_and_probs(self, prefix: list) -> (List[str], List[float]):
        """
        Возвращает список слов, которые могут идти после prefix,
        а так же список вероятностей этих слов
        """

        next_words, probs = [], []
        prefix = prefix[-self.n:]
        key = tuple(prefix)
        if key not in self.prefix_counters:
            return [], []
        counter = self.prefix_counters[key] 
        next_words = counter.keys()
        total = self.prefix_totals[key]

        probs = [count / total for count in counter.values()]

        return next_words, probs

In [34]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2, corpus_name="dummy")

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (2 балла).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. После этого можно добавить опцию генерации нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [133]:
import heapq
from copy import deepcopy
import math

class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def beam_search_extend(self, text, n_gram_model, n_words, beam_width):
        beam = [(0.0, text)]
        
        for _ in range(n_words):
            candidates = []   
            for logp, seq in beam:
                next_words, probs = n_gram_model.get_next_words_and_probs(seq)
                for word, p in zip(next_words, probs):
                    if p < 1e-2:
                        continue
                    if word == '<e>':
                        continue
                    new_seq = seq + [word]
                    new_logp = logp + math.log(p)
                    candidates.append((new_logp, new_seq))
            
            beam = heapq.nlargest(beam_width, candidates, key=lambda x: x[0])
        
        return beam

    def suggest_text(self, text: list, n_words=3, n_texts=1, method='beam') -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[srt]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """

        suggestions = []

        words, probs = word_completor.get_words_and_probs(text[-1])
        if len(words) == 0:
            return []
        text[-1] = words[np.array(probs).argmax()]

        if method == 'beam':
            texts_n_probs = self.beam_search_extend(text, self.n_gram_model, n_words, n_texts)
            if len(texts_n_probs):
                return [text[-n_words-1:] for prob, text in texts_n_probs]
            else:
                return [[text[-1]]]
        


In [134]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2, corpus_name="dummy")
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

## Бонусная часть: Добавляем UI

Запускать ячейки в юпитере – это хорошо, но будет лучше, если вашим решением действительно можно будет пользоваться. Для этого вам предлагается добавить полноценных User Interface. Мы рекомендуем использовать для этого [reflex](https://github.com/reflex-dev/reflex). Это Python библиотека для создания web-интерфейсом с очень богатым функционалом.

Ваша задача – сделать поле для текстового ввода, при наборе текста в котором будут появляться подсказки в реальном времени. Продумайте, как пользователь будет выбирать подсказки, сколько продолжений рекомендавать и так далее. В общем, должно получиться красиво и удобно. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным. 

За это задание можно получить до __5-ти бонусных баллов__ в зависимости о того, насколько хорошо и удобно у вас получилось. При сдаче задания прикрепите небольшой __отчет__ (полстраницы) с описанием вашей системы, а также __видео__ (1-2 минуты) с демонстрацией работы интерфейса.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

In [135]:

word_completor = WordCompletor(corpus)
n_gram_model = NGramLanguageModel(corpus=corpus, n=2, corpus_name="email")
text_suggestion = TextSuggestion(word_completor, n_gram_model)





In [140]:
input_text = "Hemrop"
translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
no_punct = input_text.translate(translator)
tokens = word_tokenize(no_punct, language='english')
tokens = [tok.lower() for tok in tokens if tok.isalpha()]
[' '.join(tokens[:-1]) + (' ' if tokens[:-1] else '') + ' '.join(text) for text in text_suggestion.suggest_text(['<s>'] * (n_gram_model.n-1) + tokens, n_words=3, n_texts=5)]

[]